## Generating 'average waiting time in units service time' tables

This notebook uses the multi thread engine of the OpenQTSim package to start up multiple processes for the parallel processing of a list of tasks. 

This particular example notebook produces the typical 'average waiting time in units service time' tables that are presented in the PIANC 2014 guideline for example. You can experiment with different distributions for the interarrival and service times. Currently implemented are:
* 'M': Exponential distribution, 
* 'Ek': Erlang k distributions (where k needs to be an integer number, e.g. E2, E3 etc).

NB: the table becomes more accurate when a larger number of arrivals (nr_arr) is simulated. Obviously there is a tradeoff between accuracy and calculation time.

### Create task list and perform calculations

In [1]:
%%time
if __name__ == '__main__': # protect your program's entry point
    
    import pandas as pd
    import numpy as np
    import openqtsim

    from multiprocessing import Pool
    from openqtsim.mt_engine import worker, Task

    # every instance will be run with nr_arr number of customers
    nr_arr = 20_000   # nr of customers to simulate
    A = 'M'           # distribution of inter arrival times (M: exponential, E2: erlang 2)
    S = 'M'           # distribution of service times (M: exponential, E2: erlang 2)
    nr_servers = 3    # nr of servers to consider (1 to nr_servers)

    # --- servers to loop through ---
    # make an array of the nr of servers to loop through
    servers =  np.array(range(1, nr_servers + 1))

    # --- utilisations to loop through ---
    # lambdas are selected to be ranging from 1 to 9
    lambdas = np.array(range(1,10))

    # mu is selected to always be 10 
    mu=10
    
    # the lambdas divided by mu provide the utilisations to loop through
    utilisations = lambdas / mu

    tasks=[]
    # one by one add tasks to the 'tasks' list
    for index, lam in enumerate(lambdas):
        for c in servers:
            tasks.append(Task(A, S, c, nr_arr, lam, mu))

    # select nr of processes that will carry out tasks submitted to Pool
    p = Pool(4)
    
    # map the worker with tasks list to the Pool 
    result = p.map(worker, tasks)
    
    # close processes in the Pool
    p.close()

Wall time: 48.3 s


### Reshape 'result' variable to the desired table format

In [2]:
# organise results into a table of waiting times as a factor of service time
df = pd.DataFrame(tasks)
df['result'] = result
df['utilisation'] = df.lam / df.mu

df.pivot('utilisation', 'c', values='result')

c,1,2,3
utilisation,,,
0.1,0.113653,0.008653,0.001078
0.2,0.240945,0.045333,0.008395
0.3,0.429421,0.099351,0.031443
0.4,0.679882,0.192654,0.085500
0.5,0.952446,0.326034,0.149725
0.6,1.432075,0.537770,0.284155
0.7,2.194788,1.011606,0.548988
0.8,4.624776,1.748601,1.090494
0.9,9.033923,3.616042,2.153340
